In [3]:
import pandas as pd
import re
import ipaddress
from urllib.parse import urlparse, urlencode
import ast
import requests
import whois
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

In [4]:
import requests
import ast
from urllib.parse import urlparse
from datetime import datetime

def extract_features(url):
    features = []

    # Fetch DNS record with retries
   
   #try:
    #    domain_name = fetch_dns_record(urlparse(url).netloc)
    #except Exception as e:
     #   print(f"Error fetching DNS record for {url}. Exception: {e}")
      #  domain_name = {}

    # Make HTTP request with retries
    response = get_http_response(url)
    if response is not None:
        # Process response
        pass
    
    # Address bar based features
    features.append(get_domain(url))
    features.append(having_ip(url))
    features.append(have_at_sign(url))
    features.append(get_length(url))
    features.append(get_depth(url))
    features.append(redirection(url))
    features.append(http_domain(url))
    features.append(tiny_url(url))
    features.append(prefix_suffix(url))

    # Domain based features
    dns = 0
    #if domain_name and domain_name.get('domain_created'):
     #   try:
      #      dns = ast.literal_eval(get_dns_record(urlparse(url).netloc))
       # except:
        #    pass
    features.append(dns)
    features.append(web_traffic(url))
    features.append(domain_age(url))
    features.append(domain_end(url))

    # HTML & Javascript based features
    if response is not None:
        features.append(iframe(response))
        features.append(mouse_over(response))
        features.append(right_click(response))
        features.append(forwarding(response))
    else:
        # If response is None, set these features to 0 or None
        features.extend([0, 0, 0, 0])

    features.append(fuzzy_hashing(url))
    features.append(perceptual_hashing(url))

    return features


In [5]:
# 1.Domain of the URL (Domain) 
def get_domain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [6]:
# 2.Checks for IP address in URL (Have_IP)
def having_ip(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [7]:
# 3.Checks the presence of @ in URL (Have_At)
def have_at_sign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

In [8]:
# 4.Finding the length of URL and categorizing (URL_Length)
def get_length(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [9]:
# 5.Gives number of '/' in URL (URL_Depth)
def get_depth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [10]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [11]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def http_domain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [12]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tiny_url(url):#listing shortening services
    shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0


In [13]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefix_suffix(url):
    if '-' in urlparse(url).netloc:
        return 1            
    else:
        return 0 

In [14]:
import requests
from requests.exceptions import ConnectionError, Timeout
from retrying import retry
import whois

# Define retry decorator
@retry(stop_max_attempt_number=3, wait_fixed=1000)  # Retry 3 times with a fixed interval of 1 second
# Function to fetch DNS record
def fetch_dns_record(url):
    try:
        domain_info = whois.whois(url)
        return domain_info
    except AttributeError as e:
        print(f"AttributeError occurred while fetching DNS record for {url}. Exception: {e}")
        return {}
    except Exception as e:
        print(f"Error fetching DNS record for {url}. Exception: {e}")
        return {}

In [15]:
import whois
import datetime
from urllib.parse import urlparse

def get_dns_record(url):
    parsed_url = urlparse(url.strip())
    domain = parsed_url.netloc

    try:
        domain_w = whois.whois(domain)
        if isinstance(domain_w, dict) and domain_w.get('creation_date') is not None:
            dns_data = {
                "domain_created": domain_w.creation_date,
                "domain_expires": domain_w.expiration_date,
                "registrar": domain_w.registrar,
                "name_servers": domain_w.name_servers
            }
        else:
            dns_data = {}
    except AttributeError as e:
        print(f"AttributeError occurred while fetching DNS record for {domain}. Exception: {e}")
        dns_data = {}
    except Exception as e:
        print(f"Error occurred while fetching DNS record for {domain}. Exception: {e}")
        dns_data = {}

    return dns_data

In [16]:
from requests.exceptions import ReadTimeout
from requests.exceptions import SSLError

def get_http_response(url):
    try:
        response = requests.get(url, timeout=10)  # Increased timeout to 10 seconds
        return response
    except ConnectionError as e:
        print(f"ConnectionError: Failed to make HTTP request to {url}. Exception: {e}")
        return None
    except SSLError as e:
        print(f"SSLError occurred while fetching URL: {url}. Exception: {e}")
        return None
    except ReadTimeout as e:
        print(f"ReadTimeout: Request timed out while reading response from {url}. Exception: {e}")
        return None


In [17]:
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup

In [40]:
import requests
import re
import time

def web_traffic(url):
    # Remove the "http://" or "https://" prefix
    url = re.sub(r'^https?://', '', url)

    # Remove any subdomains and trailing slashes
    url = re.sub(r'^www\.|/.*$', '', url)
    print(url)
    try:
        querystring = {"domain": url}
        headers = {
            "X-RapidAPI-Key": "1410e751edmsh95fb4841d316d80p1a11d1jsn09fd2fd8993d",
            "X-RapidAPI-Host": "similar-web.p.rapidapi.com"
        }
        response = requests.get("https://similar-web.p.rapidapi.com/get-analysis", headers=headers, params=querystring)
        data = response.json()
        rank = data.get('GlobalRank', {}).get('Rank')  # Get rank safely
        if rank is None:
            return 1  # Return 1 if rank is None
        rank = int(rank)
    except (requests.exceptions.RequestException, ValueError, KeyError):
        rank = 1

    if rank < 100:
        time.sleep(1)
        return 0
    else:
        time.sleep(1)
        return 1

cricbuzz.com


1

In [19]:
import whois
import datetime

def domain_age(url):
    try:
        # Get creation date of the website
        domain = url.strip().split('//')[-1].split('/')[0]
        w = whois.whois(domain)
        
        # Check if creation_date is a list and select the first element
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        
        now = datetime.datetime.now()
        age = now - creation_date
        age_in_years = age.days / 365  # Convert days to years
        print(age_in_years)
        
        # Assign label based on age
        if age_in_years > 1:
            label = 0
        else:
            label = 1
        
        return label
    except Exception as e:
        print(f"Error: {e}")
        return 1

#url = "https://www.cricbuzz.com/"
#domain_age(url)

In [20]:
import whois

def domain_end(url):
    try:
        domain = url.strip().split('//')[-1].split('/')[0]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        
        # Check if expiration_date is a list and select the first element
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        
        print(expiration_date)
        
        today = datetime.datetime.now()
        end = abs((expiration_date - today).days)
        
        if (end / 30) < 3:
            end = 1
        else:
            end = 0
        
        return end
    except Exception as e:
        print(f"Error: {e}")
        return 1

#url = "https://www.cricbuzz.com/"
#domain_age(url)

In [21]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 0
  else:
      soup = BeautifulSoup(response.content, 'html.parser')
      iframes = soup.find_all('iframe')
      if len(iframes) > 0:
          return 1
      else:
          return 0

In [22]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouse_over(response): 
  if response == "" :
    return 0
  else:
      soup = BeautifulSoup(response.content, 'html.parser')
      elements_with_onmouseover = soup.find_all(lambda tag: tag.has_attr('onmouseover'))
      if len(elements_with_onmouseover) > 0:
          return 1
      else:
          return 0

In [23]:
# 17.Checks the status of the right click attribute (Right_Click)
def right_click(response):
  if response == "":
    return 0
  else:
      soup = BeautifulSoup(response.content, 'html.parser')
      elements_with_oncontextmenu = soup.find_all(lambda tag: tag.has_attr('oncontextmenu'))
      if len(elements_with_oncontextmenu) > 0:
          return 1  # Found elements with oncontextmenu
      else:
          return 0

In [24]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
        return 1

In [42]:
import ppdeep
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import csv
import os
from pathlib import Path
import pandas as pd
import numpy as np
import ppdeep
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import csv
import os
from pathlib import Path
import pandas as pd
import numpy as np

def csv_r_w(csv_path):
    with open(csv_path) as csvfile:
        reader = csv.DictReader(csvfile)
        rows = []  # Store rows to write back to CSV
        for row in reader:
            url = row["url"]
            try:
                response = requests.get(url, timeout=10)  # Adjust timeout as needed
                if response.status_code == 200:
                    # Parse the HTML content using BeautifulSoup
                    soup = BeautifulSoup(response.content, "html.parser")
                    html = soup.prettify()
                    body_content = soup.find('body').prettify()  # Extract only the body content
                    h = ppdeep.hash(body_content)
                    row["fuzzy_hash_value"] = str(h)
                else:
                    print(f"Failed to load the URL: {url}")
            except requests.exceptions.Timeout:
                print(f"Timeout occurred while accessing URL: {url}")
                continue
            except requests.exceptions.RequestException as e:
                print(f"An error occurred while accessing URL: {url}: {e}")
                continue
            
            rows.append(row)  # Add row to list

    # Define the fieldnames for writing the CSV file
    fieldnames = ["url", "hash_value", "img_path", "html", "fuzzy_hash_value"]

    # Write the updated rows back to the CSV file
    with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)  # Write all rows back to the CSV file

legit_csv_path = r"C:\Users\R\Downloads\legitimate_urls.csv"
phish_csv_path = r"C:\Users\R\Downloads\phishing_urls.csv"

csv_r_w(legit_csv_path)
csv_r_w(phish_csv_path)



def load_fuzzy_hash_values(file_url):
    fuzzy_hash_values = []
    with open(file_url, newline='', encoding='utf-8') as csvfile:
        # Set the maximum field size explicitly
        csv.field_size_limit(1000000)  # Adjust the value as needed
        reader = csv.DictReader(csvfile)
        for row in reader:
            fuzzy_hash_values.append(row["fuzzy_hash_value"])
    return fuzzy_hash_values

# Load fuzzy hash values from CSV file
legit_fuzzy_hash_values = load_fuzzy_hash_values(r"C:\Users\R\Downloads\legitimate_urls.csv")
phish_fuzzy_hash_values = load_fuzzy_hash_values(r"C:\Users\R\Downloads\phishing_urls.csv")


def fuzzy_hashing(url):
    test_hash = ""  # Default value
    # Send a GET request to the test URL
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        html = soup.prettify()
        body_content = soup.find('body').prettify()  # Extract only the body content
        test_hash = ppdeep.hash(body_content)
          # Assign a value here
    else:
        print(f"Failed to load the URL: {url}")
    # Calculate ratios and store in a list
    legit_ratios = []
    for fuzzy_hash_value in legit_fuzzy_hash_values:
        ratio = fuzz.ratio(test_hash, fuzzy_hash_value)
        legit_ratios.append(ratio)
    # Compute the average ratio
    legit_average_ratio = np.mean(legit_ratios)
    # Calculate ratios and store in a list
    phish_ratios = []
    for fuzzy_hash_value in phish_fuzzy_hash_values:
        ratio = fuzz.ratio(test_hash, fuzzy_hash_value)
        phish_ratios.append(ratio)

    # Compute the average ratio
    phish_average_ratio = np.mean(phish_ratios)
    print(phish_average_ratio)
    print(legit_average_ratio)
    if phish_average_ratio > legit_average_ratio:
        label = 1
    else:
        label = 0
    return label


Timeout occurred while accessing URL: https://www.religionenlibertad.com


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to load the URL: http://www.ipinfo.io


In [44]:
from PIL import Image
import imagehash
import csv
import os
import imagehash
from PIL import Image
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from pathlib import Path
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

csv.field_size_limit(1000000)  

def csv_r_w(csv_path,dest_path):
    # Set the destination directory
    destination = Path(dest_path)
    destination.mkdir(parents=True, exist_ok=True)

    # Create a new Edge browser instance
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service)
    driver.maximize_window()
    # Set the start number for the screenshot name
    start_number = 1
    # Read the URLs from a CSV file
    with open(csv_path) as csvfile:
        reader = csv.DictReader(csvfile)
        rows = []  # Store rows to write back to CSV
        for row in reader:
            url = row["url"]

            # Navigate to the URL
            driver.get(url)
            # Wait for up to 60 seconds for the page to load completely
            WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # Wait for an additional 2 seconds to ensure all content is rendered
            time.sleep(2)

            # Take a screenshot and save it to a file
            filename = f"screenshot_{start_number}.png"
            filepath = destination / filename
            driver.save_screenshot(str(filepath))

            # Calculate the hash value for the screenshot
            legit_hash = imagehash.average_hash(Image.open(filepath))

            # Save the hash value and image path to the CSV file
            row["hash_value"] = str(legit_hash)  # Convert hash value to string
            row["img_path"] = str(filepath)      # Convert filepath to string
            rows.append(row)                      # Add row to list

            # Increment the start number
            start_number += 1

    # Write the updated rows back to the CSV file
    with open(csv_path, "w", newline="") as csvfile:
        fieldnames = ["url", "hash_value", "img_path", "html", "fuzzy_hash_value"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)  # Write all rows back to the CSV file

    # Close the browser
    driver.quit()

legit_path = r"C:\Users\R\Downloads\phishing\screenshot_legitimate"
legit_csv_path = r"C:\Users\R\Downloads\legitimate_urls.csv"
csv_r_w(legit_csv_path,legit_path)

phish_path = r"C:\Users\R\Downloads\phishing\screenshot_phish"
phish_csv_path = r"C:\Users\R\Downloads\phishing_urls.csv"
csv_r_w(phish_csv_path,phish_path)


def perceptual_hashing(url):
    # Set the destination directory
    destination = Path(r"C:\Users\R\Downloads\phishing\screenshot_test")
    destination.mkdir(parents=True, exist_ok=True)

    # Set the start number for the screenshot name
    start_number = 1

    # Create a new Edge browser instance
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service)
    driver.maximize_window()
    driver.get(url)
    # Wait for up to 60 seconds for the page to load completely
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Wait for an additional 2 seconds to ensure all content is rendered
    time.sleep(2)

    # Take a screenshot and save it to a file
    test_filename = f"screenshot_{start_number}.png"
    test_filepath = destination / test_filename
    driver.save_screenshot(str(test_filepath))

    # Increment the start number
    start_number += 1

    # Close the browser
    driver.quit()
    test_hash = imagehash.average_hash(Image.open(test_filepath))
    print(test_hash)
    # Function to calculate the average hash difference
    def calculate_avg_hash_diff(test_hash, url_list):
        total_diff = 0
        count = 0
        for hash_value in url_list['hash_value']:
            if not pd.isnull(hash_value):  # Skip NaN values
                total_diff += imagehash.hex_to_hash(test_hash) - imagehash.hex_to_hash(hash_value)
                count += 1
        if count > 0:
            return total_diff / count
        else:
            return None

    # Load the datasets
    def load_datasets(file_url, sample_size):
        urls = pd.read_csv(file_url)
        urls = urls.sample(n=sample_size)
        return urls

    # Set paths and initialize driver
    edge_driver_path = EdgeChromiumDriverManager().install()
    destination = Path(r"C:\Users\R\Downloads\phishing")

    # Calculate test hash
    with webdriver.Edge(service=Service(edge_driver_path)) as driver:
        driver.get(url)
        # Wait for up to 60 seconds for the page to load completely
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Wait for an additional 2 seconds to ensure all content is rendered
        time.sleep(2)
        test_filepath = destination / "screenshot_test" / "screenshot_1.png"
        driver.save_screenshot(str(test_filepath))
        test_hash = str(imagehash.average_hash(Image.open(test_filepath)))

    # Load datasets
    phish_url_list = load_datasets(destination / "phishing_urls.csv", 100)
    legit_url_list = load_datasets(destination / "legitimate_urls.csv", 100)

    # Calculate average hash differences
    avg_phish_hash_diff = calculate_avg_hash_diff(test_hash, phish_url_list)
    avg_legit_hash_diff = calculate_avg_hash_diff(test_hash, legit_url_list)

    # Print results
    print("Average difference between test hash and phishing URLs:", avg_phish_hash_diff)
    print("Average difference between test hash and legitimate URLs:", avg_legit_hash_diff)

    if avg_phish_hash_diff > avg_legit_hash_diff :
        label = 0
    else :
        label =1
    return label

In [45]:
# Load the datasets
def load_datasets(file_url, sample_size):
    urls = pd.read_csv(file_url)
    urls = urls.sample(n=sample_size)
    return urls

In [47]:
# Preprocess URLs
url_list = load_datasets(r"C:\Users\R\Downloads\phishing_urls.csv", 100)
legitimate_urls = load_datasets(r"C:\Users\R\Downloads\legitimate_urls.csv", 100)

url_list = url_list[["url"]]
legitimate_urls = legitimate_urls[["url"]]

In [48]:
url_list

,url
12,https://ecopacking.com.ua/
19,https://weshop-sfr.com/password
15,https://urlscan.io/
63,http://www.doable.nl
46,http://www.is-a-soxfan.org
...,...
17,https://teams.live.com/dl/launcher/launcher.ht...
69,http://www.jfalling.com
50,http://www.pranazfinance.com
10,https://axieinfinlty.pages.dev/


In [49]:
legitimate_urls

,url
89,https://www.pimpim.lt
47,https://www.nnlm.gov/podcast
1,https://archive.org/details/universallibrary?a...
40,https://www.youtube.com/user/NIDDKgov
56,https://www.ibm.com/application-delivery?lnk=f...
...,...
93,https://www.medicalplasticsnews.com
6,https://medium.com/@Manda_like_wine/this-is-fo...
77,https://www.ainewsletter.com
28,https://www.google.com/


In [50]:
# Combine the datasets
urls = pd.concat([url_list, legitimate_urls]).reset_index(drop=True)

In [51]:
urls

,url
0,https://ecopacking.com.ua/
1,https://weshop-sfr.com/password
2,https://urlscan.io/
3,http://www.doable.nl
4,http://www.is-a-soxfan.org
...,...
195,https://www.medicalplasticsnews.com
196,https://medium.com/@Manda_like_wine/this-is-fo...
197,https://www.ainewsletter.com
198,https://www.google.com/


In [52]:
# Add the actual label
labels = pd.Series([1]*len(urls[:100]) + [0]*len(urls[100:]), name="Label")
urls = pd.concat([urls, labels], axis=1) 
#phising - 1
#legitimate - 0

In [53]:
urls

,url,Label
0,https://ecopacking.com.ua/,1
1,https://weshop-sfr.com/password,1
2,https://urlscan.io/,1
3,http://www.doable.nl,1
4,http://www.is-a-soxfan.org,1
...,...,...
195,https://www.medicalplasticsnews.com,0
196,https://medium.com/@Manda_like_wine/this-is-fo...,0
197,https://www.ainewsletter.com,0
198,https://www.google.com/,0


In [54]:
print(urls.columns)

Index(['url', 'Label'], dtype='object')


In [55]:
print(urls.head())

                               url  Label
0       https://ecopacking.com.ua/      1
1  https://weshop-sfr.com/password      1
2              https://urlscan.io/      1
3             http://www.doable.nl      1
4       http://www.is-a-soxfan.org      1


In [56]:
print(urls.iloc[83])

url      http://www.doctorww.com
Label                          1
Name: 83, dtype: object


In [57]:
columns = [
    'Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth',
    'Redirection', 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'dns', 'Web_Traffic', 'Domain_Age', 'Domain_End', 'iFrame',
    'Mouse_Over', 'Right_Click', 'Web_Forwards', 'Fuzzy_Hashing', 'Perceptual_Hashing', 'Label']
all_features=[]

In [58]:
with ThreadPoolExecutor(max_workers=5) as executor:
    for _, row in urls.iterrows():
        url = row["url"]
        label = row["Label"]
        #domain_name = get_dns_record(url)
        features = extract_features(url)
        
        features.append(label)
        print(features)
        all_features.append(features)
        print(all_features)
        # Add features to DataFrame
        #extracted_features.loc[_] = features


ecopacking.com.ua
0.01643835616438356
2025-04-08 16:39:03
11.18867924528302
11.858585858585858
ffff00644c60e0c0
Average difference between test hash and phishing URLs: 32.22
Average difference between test hash and legitimate URLs: 32.11
['ecopacking.com.ua', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[['ecopacking.com.ua', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]]
weshop-sfr.com
0.01643835616438356
2025-04-08 13:10:27
10.849056603773585
11.88888888888889
0083ffffff7d6100
Average difference between test hash and phishing URLs: 31.41
Average difference between test hash and legitimate URLs: 31.53
['weshop-sfr.com', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]
[['ecopacking.com.ua', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], ['weshop-sfr.com', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]]
urlscan.io
8.158904109589042
2025-02-17 22:43:23
11.29245283018868
11.797979797979798
00fffbffffffffff
Average difference between tes

2024-04-14 17:36:06,215 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


Error: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'


2024-04-14 17:36:06,721 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


None
Error: unsupported operand type(s) for -: 'NoneType' and 'datetime.datetime'
16.433962264150942
10.696969696969697
00ffffefffffffc3
Average difference between test hash and phishing URLs: 26.51
Average difference between test hash and legitimate URLs: 26.65
['fotos.lc', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
[['ecopacking.com.ua', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], ['weshop-sfr.com', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], ['urlscan.io', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], ['doable.nl', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1], ['is-a-soxfan.org', 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1], ['im-creator.com', 0, 0, 1, 3, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1], ['linkby.tw', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], ['mouserunner.net', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], ['bellaitaliatour.com', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

2024-04-14 17:57:26,101 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out


Error: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'


2024-04-14 17:57:36,719 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out


None
Error: unsupported operand type(s) for -: 'NoneType' and 'datetime.datetime'
11.726415094339623
10.93939393939394
3f3fffffffffffff
Average difference between test hash and phishing URLs: 25.87
Average difference between test hash and legitimate URLs: 24.37
['ulsm.pt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
[['ecopacking.com.ua', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], ['weshop-sfr.com', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], ['urlscan.io', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], ['doable.nl', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1], ['is-a-soxfan.org', 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1], ['im-creator.com', 0, 0, 1, 3, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1], ['linkby.tw', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], ['mouserunner.net', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], ['bellaitaliatour.com', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [59]:
extracted_features = pd.DataFrame(all_features, columns=columns)

In [60]:
extracted_features

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,dns,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Fuzzy_Hashing,Perceptual_Hashing,Label
0,ecopacking.com.ua,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1
1,weshop-sfr.com,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1
2,urlscan.io,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,doable.nl,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1,1
4,is-a-soxfan.org,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,christopher-cloos.com,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1
62,winnicapolska.com,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
63,onlyidea.com,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1
64,100dof.com,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1


In [220]:
import pandas as pd
csv_file_path = (r"C:\Users\R\Downloads\phishing\DataFilesfile.csv")
extracted_features.to_csv(csv_file_path, index=False)
# Print a message to confirm that the CSV file has been saved
print(f"DataFrame has been successfully exported to {csv_file_path}")

DataFrame has been successfully exported to C:\Users\R\Downloads\phishing\DataFilesfile.csv
